In [1]:
import torch
import cv2
import os
import glob

# Load the model
model = torch.load('./pytorch/torch_alex.pth')
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [2]:
import numpy as np
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam import GradCAM, LayerCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image


# Define transforms
transform_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the size your model expects
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])  # Normalize (these values are for pre-trained models on ImageNet)
])

# Define the inverse of the normalization
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

# Load image
images_folder_path = "./pytorch/data4/"
# images_folder_path = "./image/"
jpg_files = glob.glob(os.path.join(images_folder_path, "*.jpg"))
image_names = [os.path.basename(file) for file in jpg_files]

print(f'image names: {image_names}')
image_paths = []
for i in range(len(image_names)):
    image_paths.append(images_folder_path+image_names[i])
    
print(f'image paths: {image_paths}')
images = [Image.open(image_path).convert('RGB') for image_path in image_paths]

# Apply transformation to fit the model
input_images = [transform_pipeline(image) for image in images]

# Define target classes based on model's output
target_classes = []

# put the image into the model
if torch.cuda.is_available():
    model = model.cuda()

for i, input_image in enumerate(input_images):
    # Add a batch dimension to the image
    input_image = input_image.unsqueeze(0)
    # If CUDA is available, move the image to GPU
    if torch.cuda.is_available():
        input_image = input_image.cuda()

    # Pass the image through the model
    output = model(input_image)
    # print(f"The output of model is {probabilities}")
    # Convert the output to probabilities using softmax
    probabilities = torch.nn.functional.softmax(output[0], dim=-1)
    
    # Get the predicted class by finding the class with the highest probability
    _, predicted_class = torch.max(probabilities, dim=-1)

    label = 'dog' if predicted_class == 1 else 'cat'
    print(f'model guess {label} from {image_names[i]} ')
    # Check if both classes have a probability higher than 0.5
    if probabilities[0] > 0.5 and probabilities[1] > 0.5:
        predicted_class = 2  # Both
    target_classes.append(predicted_class.item())

print(f"Target classes: {target_classes}")

# The last Conv layer in AlexNet
print(model.features[10])
target_layers = [model.features[10]]

for i, (input_image, target_class) in enumerate(zip(input_images, target_classes)):
    print(f"analysis {i} times")
    # Add a batch dimension to the image
    input_image_unnorm = input_image.unsqueeze(0)

    # Construct the CAM object
    grad_cam = GradCAM(model=model, target_layers=target_layers)
    layer_cam = LayerCAM(model=model, target_layers=target_layers)
    # Define the target
    targets = [ClassifierOutputTarget(target_class)]
    # Generate the Grad-CAM and LayerCAM
    grayscale_gradcam = grad_cam(input_tensor=input_image_unnorm, targets=targets)
    grayscale_layercam = layer_cam(input_tensor=input_image_unnorm, targets=targets)
    # grayscale_cam has only one image in the batch:
    grayscale_gradcam = grayscale_gradcam[0, :]
    grayscale_layercam = grayscale_layercam[0, :]
    # Apply the inverse normalization to the image
    input_image = inv_normalize(input_image)
    # Make sure the image values are in the range [0, 1]
    input_image = input_image.clamp(0, 1)
    # Convert the tensor image to a numpy array
    input_image = input_image.permute(1, 2, 0).detach().cpu().numpy()
    
    # Visualization 
    visualization_gradcam = show_cam_on_image(input_image, grayscale_gradcam, use_rgb=True)
    visualization_layercam = show_cam_on_image(input_image, grayscale_layercam, use_rgb=True)
    
    # Save GradCam Image
    filename = os.path.join('./gradcam/', f'Grad-CAM_{image_names[i]}')
    filename2 = os.path.join('./layercam/', f'LayerCAM_{image_names[i]}')
    print(f"Saving GradCam Image to {filename}")
    print(f"Saving LayerCAM Image to {filename2}")
    cv2.imwrite(filename, visualization_gradcam)
    cv2.imwrite(filename2, visualization_layercam)

image names: ['both.jpg', 'cat.jpg', 'dog.jpg']
image paths: ['./pytorch/data4/both.jpg', './pytorch/data4/cat.jpg', './pytorch/data4/dog.jpg']
model guess cat from both.jpg 
model guess dog from cat.jpg 
model guess dog from dog.jpg 
Target classes: [0, 1, 1]
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
analysis 0 times
Saving GradCam Image to ./gradcam/Grad-CAM_both.jpg
Saving LayerCAM Image to ./layercam/LayerCAM_both.jpg
analysis 1 times
Saving GradCam Image to ./gradcam/Grad-CAM_cat.jpg
Saving LayerCAM Image to ./layercam/LayerCAM_cat.jpg
analysis 2 times
Saving GradCam Image to ./gradcam/Grad-CAM_dog.jpg
Saving LayerCAM Image to ./layercam/LayerCAM_dog.jpg
